### Random Forest Guided Example

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421097 entries, 0 to 421096
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(85), object(26)
memory usage: 356.6+ MB


In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [5]:
handle_missing = pd.Series()
for column in y2015.columns:
    if len(y2015[y2015[column].isnull()]) < (0.3 * 421095):
        handle_missing = pd.concat([handle_missing, y2015[column]], axis = 1)  
handle_missing.head()        

,0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,NaN,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,NaN,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,NaN,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,NaN,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,NaN,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [6]:
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')
y2015_numeric = y2015.select_dtypes(include = ['float64'])
y2015_categorical = y2015.select_dtypes(include = 'object')
y2015_categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421097 entries, 0 to 421096
Data columns (total 25 columns):
id                           421097 non-null object
term                         421095 non-null object
grade                        421095 non-null object
sub_grade                    421095 non-null object
emp_title                    397221 non-null object
emp_length                   397278 non-null object
home_ownership               421095 non-null object
verification_status          421095 non-null object
issue_d                      421095 non-null object
loan_status                  421095 non-null object
pymnt_plan                   421095 non-null object
url                          421095 non-null object
desc                         45 non-null object
purpose                      421095 non-null object
title                        420963 non-null object
zip_code                     421095 non-null object
addr_state                   421095 non-null object
earliest_

In [7]:
y2015_numeric_filled = y2015_numeric.fillna(y2015_numeric.mean())

Lets drop the first two columns and the last two rows from our dataset.

In [8]:
y2015_numeric_filled = y2015_numeric_filled.iloc[:-2,2:]

In [9]:
y2015_numeric_filled.head()

,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,0.0,33.000000,2.000000,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,0.0,34.023391,66.592609,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,1.0,34.023391,66.592609,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,0.0,42.000000,66.592609,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,4.0,34.023391,30.000000,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


I have used Principal component analysis to select numerical features for our model.

In [10]:
pca = PCA(n_components= 5, whiten=True)
X = pca.fit(y2015_numeric_filled).transform(y2015_numeric_filled)
X = pd.DataFrame(data = X, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

Next is selecting categorical features to include in our model.I have used barplots to visualize the features with higher impact to the target variable 'loan status'.But first lets drop the missing columns because we have enough features to try our model.if the performance is really low we will try to improve it by imputing the missing values and rerun the model again.For now lets just drop the columns with missing values. 

I decided to drop those with missing values because i have enough number of variables with no missing values.Therefore i will include all variables with only two missing values(i.e 421095 non-null objects and two null objects) and then remove the two missing values.

In [11]:
y2015_categorical = y2015_categorical[:-2]
y2015_categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421095 entries, 0 to 421094
Data columns (total 25 columns):
id                           421095 non-null object
term                         421095 non-null object
grade                        421095 non-null object
sub_grade                    421095 non-null object
emp_title                    397221 non-null object
emp_length                   397278 non-null object
home_ownership               421095 non-null object
verification_status          421095 non-null object
issue_d                      421095 non-null object
loan_status                  421095 non-null object
pymnt_plan                   421095 non-null object
url                          421095 non-null object
desc                         45 non-null object
purpose                      421095 non-null object
title                        420963 non-null object
zip_code                     421095 non-null object
addr_state                   421095 non-null object
earliest_

In [12]:
# dependent_var = []
# for var in y2015_categorical.columns:
#     contingency_table = pd.crosstab(
#                           y2015_categorical[var],
#                           y2015_categorical['loan_status'],
#                           margins = True
#                          )
#     g, p, dof, expctd = chi2_contingency(contingency_table)
#     if p < 0.05:
#         dependent_var.append(var)
# dependent_var        

In [13]:
categorical_features = y2015_categorical.drop(['loan_status', 'zip_code', 'earliest_cr_line', 'id',
                                               'emp_title', 'revol_util', 'sub_grade','url', 'addr_state'], axis = 1)
# dependent_var.remove('loan_status')
for i in categorical_features:
    column = categorical_features[i]
    print(i)
    print(column.nunique())

term
2
grade
7
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
pymnt_plan
1
desc
34
purpose
14
title
27
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [14]:
# y2015_categ_final = y2015_categorical[dependent_var]

In [15]:
rfc = ensemble.RandomForestClassifier()
X = pd.concat([X, categorical_features], axis = 1)
Y = y2015_categorical['loan_status']
X = pd.get_dummies(X)
cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

array([0.93510009, 0.91964095, 0.92013963, 0.92313172, 0.9018048 ,
       0.91695559, 0.91897219, 0.93224726, 0.92789797, 0.93675486])

we can see from the cross validated scores in the above that our model has performed quite ok.Now lets go ahead and check if we can do it without using anything related to payment amount or outstanding principal.

In [19]:
y2015_numeric_filled.columns

Index(['funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment',
       'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint',
       'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
       'mo

**Lets exclude payment amount('total_pymnt', 'total_pymnt_inv') and outstanding principal('out_prncp', 'out_prncp_inv') from our dataset and run PCA and the RF model.**

In [23]:
y2015_numeric_filled = y2015_numeric_filled.drop(['total_pymnt', 'total_pymnt_inv', 'out_prncp', 'out_prncp_inv'], axis = 1)
pca = PCA(n_components= 5, whiten=True)
Z = pca.fit(y2015_numeric_filled).transform(y2015_numeric_filled)
Z = pd.DataFrame(data = Z, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

In [22]:
rfc_1 = ensemble.RandomForestClassifier()
Z = pd.concat([X, categorical_features], axis = 1)
Y = y2015_categorical['loan_status']
Z = pd.get_dummies(X)
cross_val_score(rfc_1, Z, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

array([0.93189428, 0.92006839, 0.92545891, 0.9170288 , 0.90268345,
       0.92208502, 0.9160987 , 0.9335534 , 0.92787423, 0.75214934])

**The result above proves that yes we can do it with out payment amount and outstanding principal in our data.**